In [1]:
import pathlib
import os
import requests
from urllib.parse import urlparse, parse_qs
from io import BytesIO
import pandas as pd
import json

# city api https://public.opendatasoft.com/

In [7]:
GET_REQUEST = "https://www.eco-visio.net/api/aladdin/1.0.0/pbl/publicwebpageplus/4586?withNull=true"

In [8]:
url_endpoint = GET_REQUEST
response = requests.get(url_endpoint)
response

<Response [200]>

In [9]:
data_json = json.loads(response.content)

In [10]:
data_json[0]

{'token': None,
 'id_pdc_img': 0,
 'idPdc': 100021443,
 'lat': 69.96787,
 'lon': 23.36929,
 'nom': 'Alta_2 (NO)',
 'publicMessage': None,
 'photo': [{'lien': 'https://filer.eco-counter-tools.com/file/c3/a6b6e6ded8b909892faafb44eebf6883120ef9ab7695ccb8aca0401899b1fec3/14315298005820.jpg'}],
 'lienPublic': None,
 'pratique': [{'pratique': 2, 'id': 101021443},
  {'pratique': 2, 'id': 102021443}],
 'mainPratique': 2,
 'filtre': None,
 'formule_site': None,
 'fin': None,
 'debut': '01/01/2013',
 'finPeriode': None,
 'debutPeriode': '01/01/2021',
 'current_year_default': 1,
 'externalUrl': 'https://www.eco-compteur.com',
 'nomOrganisme': 'Bike Count Display Interactive Map',
 'logo': 'https://filer.eco-counter-tools.com/file/33/43649b3090eaa41e49d73bb31ff037efe2f97745f8a8e8eafb3baab42acf6433/1556292443839.png',
 'pays': 'fr',
 'sig': 1,
 'pictoPublic': 'totem_vert.png',
 'today': '22/09/2021',
 'total': 33193,
 'lastDay': 109,
 'moyD': 126,
 'totalLY': None,
 'lastDay8': None,
 'moyDLY': Non

In [13]:
with open('counting_stations_metadata_global.json', 'w') as outfile:
    json.dump(data_json, outfile)

In [2]:
file = open('counting_stations_metadata_global.json',)
counting_stations_metadata_global = json.load(file)

In [3]:
counting_stations_metadata_global

[{'token': None,
  'id_pdc_img': 0,
  'idPdc': 100021443,
  'lat': 69.96787,
  'lon': 23.36929,
  'nom': 'Alta_2 (NO)',
  'publicMessage': None,
  'photo': [{'lien': 'https://filer.eco-counter-tools.com/file/c3/a6b6e6ded8b909892faafb44eebf6883120ef9ab7695ccb8aca0401899b1fec3/14315298005820.jpg'}],
  'lienPublic': None,
  'pratique': [{'pratique': 2, 'id': 101021443},
   {'pratique': 2, 'id': 102021443}],
  'mainPratique': 2,
  'filtre': None,
  'formule_site': None,
  'fin': None,
  'debut': '01/01/2013',
  'finPeriode': None,
  'debutPeriode': '01/01/2021',
  'current_year_default': 1,
  'externalUrl': 'https://www.eco-compteur.com',
  'nomOrganisme': 'Bike Count Display Interactive Map',
  'logo': 'https://filer.eco-counter-tools.com/file/33/43649b3090eaa41e49d73bb31ff037efe2f97745f8a8e8eafb3baab42acf6433/1556292443839.png',
  'pays': 'fr',
  'sig': 1,
  'pictoPublic': 'totem_vert.png',
  'today': '22/09/2021',
  'total': 33193,
  'lastDay': 109,
  'moyD': 126,
  'totalLY': None,
  '

In [65]:
def get_position_of_brackets(String):
    first = String.find('(')
    second = String.find(')')
    return [first,second]

def get_country_between_brackets(String, brackets_index):
    country = String[brackets_index[0]+1: brackets_index[1]]
    return country
    
print(get_position_of_brackets("EV7_Sterzing (IT)"))

[13, 16]


In [69]:
german_counting_stations = []
for counting_station in counting_stations_metadata_global:
    name = counting_station["nom"]
    brackets_index = get_position_of_brackets(name)
    country = get_country_between_brackets(name, brackets_index)
    if country == "DE":
        german_counting_stations.append(counting_station)
print(len(german_counting_stations))
    

56
